<a href="https://colab.research.google.com/github/ssingh-rookie/tensorflow_nlp/blob/master/text_generation_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Import modules

In [0]:
import tensorflow as tf
import numpy as np
import os
import time

#### Download and create training data

In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'r').read()
vocab = sorted(set(text))
print("vocab :", vocab)

1122304/1115394 [==============================] - 0s 0us/step
vocab : ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [0]:
char_to_index = {char:index for index, char in enumerate(vocab)}
index_to_char = np.array(vocab)

vocab = sorted(set(text))
print("vocab :", vocab)

vocab : ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [0]:
text_as_int = np.array([char_to_index[char] for char in text])
text_as_int[:200]

array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
       44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39,
       52, 63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1,
       51, 43,  1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31,
       54, 43, 39, 49,  6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56,
       57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39,
       56, 43,  1, 39, 50, 50,  1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56,
       39, 58, 46, 43, 56,  1, 58, 53,  1, 42, 47, 43,  1, 58, 46, 39, 52,
        1, 58, 53,  1, 44, 39, 51, 47, 57, 46, 12,  0,  0, 13, 50, 50, 10,
        0, 30, 43, 57, 53, 50, 60, 43, 42,  8,  1, 56, 43, 57, 53, 50, 60,
       43, 42,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43,
       52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63, 53, 59])

#### Create training examples and targets

In [0]:
seq_length = 100
examples_per_epoch = len(text)

In [0]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
[index_to_char[char.numpy()] for char in char_dataset.take(5)]

['F', 'i', 'r', 's', 't']

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
for sequence in sequences.take(3):
  print(repr(" ".join(index_to_char[sequence.numpy()])))

'F i r s t   C i t i z e n : \n B e f o r e   w e   p r o c e e d   a n y   f u r t h e r ,   h e a r   m e   s p e a k . \n \n A l l : \n S p e a k ,   s p e a k . \n \n F i r s t   C i t i z e n : \n Y o u  '
'a r e   a l l   r e s o l v e d   r a t h e r   t o   d i e   t h a n   t o   f a m i s h ? \n \n A l l : \n R e s o l v e d .   r e s o l v e d . \n \n F i r s t   C i t i z e n : \n F i r s t ,   y o u   k'
"n o w   C a i u s   M a r c i u s   i s   c h i e f   e n e m y   t o   t h e   p e o p l e . \n \n A l l : \n W e   k n o w ' t ,   w e   k n o w ' t . \n \n F i r s t   C i t i z e n : \n L e t   u s   k i"


In [0]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

In [0]:
split_input_target("Sundar")

('Sunda', 'undar')

In [0]:
dataset = sequences.map(split_input_target)

In [0]:
for input, target in dataset.take(5):
  print(repr(" ".join(index_to_char[input.numpy()])))
  print(repr(" ".join(index_to_char[target.numpy()])))
  print("--------------------------------------------")

'F i r s t   C i t i z e n : \n B e f o r e   w e   p r o c e e d   a n y   f u r t h e r ,   h e a r   m e   s p e a k . \n \n A l l : \n S p e a k ,   s p e a k . \n \n F i r s t   C i t i z e n : \n Y o u'
'i r s t   C i t i z e n : \n B e f o r e   w e   p r o c e e d   a n y   f u r t h e r ,   h e a r   m e   s p e a k . \n \n A l l : \n S p e a k ,   s p e a k . \n \n F i r s t   C i t i z e n : \n Y o u  '
--------------------------------------------
'a r e   a l l   r e s o l v e d   r a t h e r   t o   d i e   t h a n   t o   f a m i s h ? \n \n A l l : \n R e s o l v e d .   r e s o l v e d . \n \n F i r s t   C i t i z e n : \n F i r s t ,   y o u  '
'r e   a l l   r e s o l v e d   r a t h e r   t o   d i e   t h a n   t o   f a m i s h ? \n \n A l l : \n R e s o l v e d .   r e s o l v e d . \n \n F i r s t   C i t i z e n : \n F i r s t ,   y o u   k'
--------------------------------------------
"n o w   C a i u s   M a r c i u s   i s   c h i e f   e n e m y   t o   t h

#### Create training batches

In [0]:
BATCH_SIZE=64
BUFFER_SIZE = 10000

In [0]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [0]:
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

#### Build The Model

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
for input_batch_sample, target_batch_sample in dataset.take(1):
  example_batch_predictions = model(input_batch_sample)
  print(example_batch_predictions.shape)

(64, 100, 65)


In [0]:
example_batch_predictions[0].shape

TensorShape([100, 65])

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [0]:
print(repr(" ".join([index_to_char[index] for index in input_batch_sample[0]])))
print(repr(" ".join([index_to_char[index] for index in sampled_indices])))

"  w e l l   p r o c e e d e d , \n T o   w a r n   f a l s e   t r a i t o r s   f r o m   t h e   l i k e   a t t e m p t s . \n I   n e v e r   l o o k ' d   f o r   b e t t e r   a t   h i s   h a n"
'J v & m a   x p O d J b F 3 $ U , Z P s J o L \n . f . b U X G v j c s f D p R   3 B o   K r W O - T c A e o a r V : : S e & K b Y y : Q T M Q j & d z A O I q a z q M B k S Q q 3 G y u H f ? R T ; f V'


#### Train the model

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 23s 135ms/step - loss: 2.6548
Epoch 2/10
172/172 [==============================] - 23s 136ms/step - loss: 1.9561
Epoch 3/10
172/172 [==============================] - 23s 136ms/step - loss: 1.6897
Epoch 4/10
172/172 [==============================] - 23s 135ms/step - loss: 1.5429
Epoch 5/10
172/172 [==============================] - 23s 135ms/step - loss: 1.4564
Epoch 6/10
172/172 [==============================] - 23s 135ms/step - loss: 1.3957
Epoch 7/10
172/172 [==============================] - 23s 134ms/step - loss: 1.3513
Epoch 8/10
172/172 [==============================] - 23s 134ms/step - loss: 1.3120
Epoch 9/10
172/172 [==============================] - 23s 134ms/step - loss: 1.2778
Epoch 10/10
172/172 [==============================] - 23s 135ms/step - loss: 1.2458


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char_to_index[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.5

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(index_to_char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"JULIET: "))

JULIET: that which I but believe not your lady:
For this way, and you shall be the vein of them.

CAMILLO:
I do not stand and will be company:
There is no with the fire, the time would set down the county
But that hath promised me in the world.

KING RICHARD III:
Stand, and do you good nor good to the duke
What is the precious lord?

KING RICHARD III:
Came to the lady, my must not know
That which the deed is worthy father, he is had done with him.

CAPULET:
How shall I be gone, so stay and with the very house.

Third Citizen:
Your mother was the battle let thy brother did the blood
With the heavens have brought you for the court.

CORIOLANUS:
I have spoke of all that hath company.

KING RICHARD III:
Son, I think there which is the worst; he is in your blows,
For this is the direct of my lord, he was she would do you shall stook and welcome.

MENENIUS:
I will show the duke, which is the way;
That shall be some mighty soul to death,
When we more death,
That she will sting the charity of 